In [63]:
# !pip install tensorflow
!pip install imblearn


In [4]:
import pandas as pd
import numpy as np 

In [65]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [146]:
data=pd.read_csv('/content/drive/MyDrive/edf_csv/amine_final.csv')

In [147]:
data

,Unnamed: 0,Unnamed: 0.1,0_Absolute energy,0_Area under the curve,0_Autocorrelation,0_Centroid,0_Entropy,0_HFD,0_Mean absolute diff,0_Mean diff,0_Median absolute diff,0_Median diff,0_Negative turning points,0_Neighbourhood peaks,0_PFD,0_Peak to peak distance,0_Positive turning points,0_SPECTRAL_ENTROPY,0_SVD_ENTROPY,0_Signal distance,0_Slope,0_Sum absolute diff,0_Total energy,0_Zero crossing rate,Tag
0,0,0.0,0.214933,0.137637,0.214933,19.908868,0.523785,0.870429,0.004001,-1.096583e-06,0.003459,0.0,2437.0,3.0,0.548537,0.010965,2459.0,0.961588,2.264482,9999.135947,4.065461e-09,40.007864,5.373869e-03,3017.0,epilepsy
1,1,1.0,0.212869,0.137648,0.212869,20.074096,0.534382,0.853697,0.004009,6.611124e-08,0.003494,0.0,2538.0,2.0,0.547341,0.010965,2541.0,0.963424,2.264026,9999.134262,-1.992912e-08,40.087510,5.322257e-03,2932.0,epilepsy
2,2,2.0,0.211065,0.132886,0.211065,20.056645,0.543723,0.883311,0.004287,2.047256e-07,0.003820,0.0,2664.0,1.0,0.545814,0.010965,2660.0,0.965067,2.242634,9999.146500,1.953178e-10,42.861512,5.277164e-03,3216.0,epilepsy
3,3,3.0,0.211173,0.130824,0.211173,19.975290,0.542275,0.902564,0.004399,2.028683e-07,0.003970,0.0,2631.0,1.0,0.546052,0.010965,2654.0,0.954357,2.230903,9999.152398,1.853369e-08,43.985484,5.279860e-03,3382.0,epilepsy
4,4,4.0,0.212760,0.132685,0.212760,19.875644,0.534411,0.898868,0.004306,1.090426e-06,0.003867,0.0,2615.0,2.0,0.546646,0.010965,2574.0,0.952564,2.239864,9999.148932,2.636620e-08,43.058994,5.319537e-03,3305.0,epilepsy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
506632,26,NaN,0.000003,0.000564,0.000003,20.354324,0.667670,0.220908,0.000002,-3.708254e-09,0.000002,0.0,1630.0,209.0,0.562039,0.000127,1640.0,0.695498,1.708365,9999.000000,-3.306952e-10,0.018259,8.325051e-08,447.0,no_epilepsy
506633,27,NaN,0.000003,0.000561,0.000003,19.680088,0.665403,0.256714,0.000002,3.998199e-09,0.000002,0.0,1671.0,225.0,0.561243,0.000120,1676.0,0.717898,1.748904,9999.000000,5.074985e-10,0.019666,7.843683e-08,429.0,no_epilepsy
506634,28,NaN,0.000003,0.000507,0.000003,18.696962,0.655014,0.221881,0.000002,-2.609512e-09,0.000002,0.0,1636.0,230.0,0.562165,0.000128,1622.0,0.725297,1.808618,9999.000000,3.268887e-10,0.019889,6.604886e-08,486.0,no_epilepsy
506635,29,NaN,0.000005,0.000654,0.000005,17.202044,0.680660,0.293953,0.000002,-7.630152e-11,0.000002,0.0,1685.0,234.0,0.561131,0.000174,1673.0,0.730297,1.704340,9999.000000,-3.416951e-10,0.021868,1.204570e-07,418.0,no_epilepsy


In [148]:
ds=data[['0_Absolute energy','0_Area under the curve','0_Autocorrelation','0_Centroid','0_Entropy','0_HFD','0_Mean absolute diff','0_Mean diff','0_Median absolute diff','0_Median diff','0_Negative turning points','0_Neighbourhood peaks','0_PFD','0_Peak to peak distance','0_Positive turning points','0_SPECTRAL_ENTROPY','0_SVD_ENTROPY','0_Signal distance','0_Slope','0_Sum absolute diff','0_Total energy','0_Zero crossing rate','Tag']]

In [149]:
ds.head()

,0_Absolute energy,0_Area under the curve,0_Autocorrelation,0_Centroid,0_Entropy,0_HFD,0_Mean absolute diff,0_Mean diff,0_Median absolute diff,0_Median diff,0_Negative turning points,0_Neighbourhood peaks,0_PFD,0_Peak to peak distance,0_Positive turning points,0_SPECTRAL_ENTROPY,0_SVD_ENTROPY,0_Signal distance,0_Slope,0_Sum absolute diff,0_Total energy,0_Zero crossing rate,Tag
0,0.214933,0.137637,0.214933,19.908868,0.523785,0.870429,0.004001,-1.096583e-06,0.003459,0.0,2437.0,3.0,0.548537,0.010965,2459.0,0.961588,2.264482,9999.135947,4.065461e-09,40.007864,0.005374,3017.0,epilepsy
1,0.212869,0.137648,0.212869,20.074096,0.534382,0.853697,0.004009,6.611124e-08,0.003494,0.0,2538.0,2.0,0.547341,0.010965,2541.0,0.963424,2.264026,9999.134262,-1.992912e-08,40.087510,0.005322,2932.0,epilepsy
2,0.211065,0.132886,0.211065,20.056645,0.543723,0.883311,0.004287,2.047256e-07,0.003820,0.0,2664.0,1.0,0.545814,0.010965,2660.0,0.965067,2.242634,9999.146500,1.953178e-10,42.861512,0.005277,3216.0,epilepsy
3,0.211173,0.130824,0.211173,19.975290,0.542275,0.902564,0.004399,2.028683e-07,0.003970,0.0,2631.0,1.0,0.546052,0.010965,2654.0,0.954357,2.230903,9999.152398,1.853369e-08,43.985484,0.005280,3382.0,epilepsy
4,0.212760,0.132685,0.212760,19.875644,0.534411,0.898868,0.004306,1.090426e-06,0.003867,0.0,2615.0,2.0,0.546646,0.010965,2574.0,0.952564,2.239864,9999.148932,2.636620e-08,43.058994,0.005320,3305.0,epilepsy


In [150]:
encode = ({'no_epilepsy': 0, 'epilepsy': 1} )
df_encoded = ds.replace(encode)

In [151]:
df_encoded

,0_Absolute energy,0_Area under the curve,0_Autocorrelation,0_Centroid,0_Entropy,0_HFD,0_Mean absolute diff,0_Mean diff,0_Median absolute diff,0_Median diff,0_Negative turning points,0_Neighbourhood peaks,0_PFD,0_Peak to peak distance,0_Positive turning points,0_SPECTRAL_ENTROPY,0_SVD_ENTROPY,0_Signal distance,0_Slope,0_Sum absolute diff,0_Total energy,0_Zero crossing rate,Tag
0,0.214933,0.137637,0.214933,19.908868,0.523785,0.870429,0.004001,-1.096583e-06,0.003459,0.0,2437.0,3.0,0.548537,0.010965,2459.0,0.961588,2.264482,9999.135947,4.065461e-09,40.007864,5.373869e-03,3017.0,1
1,0.212869,0.137648,0.212869,20.074096,0.534382,0.853697,0.004009,6.611124e-08,0.003494,0.0,2538.0,2.0,0.547341,0.010965,2541.0,0.963424,2.264026,9999.134262,-1.992912e-08,40.087510,5.322257e-03,2932.0,1
2,0.211065,0.132886,0.211065,20.056645,0.543723,0.883311,0.004287,2.047256e-07,0.003820,0.0,2664.0,1.0,0.545814,0.010965,2660.0,0.965067,2.242634,9999.146500,1.953178e-10,42.861512,5.277164e-03,3216.0,1
3,0.211173,0.130824,0.211173,19.975290,0.542275,0.902564,0.004399,2.028683e-07,0.003970,0.0,2631.0,1.0,0.546052,0.010965,2654.0,0.954357,2.230903,9999.152398,1.853369e-08,43.985484,5.279860e-03,3382.0,1
4,0.212760,0.132685,0.212760,19.875644,0.534411,0.898868,0.004306,1.090426e-06,0.003867,0.0,2615.0,2.0,0.546646,0.010965,2574.0,0.952564,2.239864,9999.148932,2.636620e-08,43.058994,5.319537e-03,3305.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
506632,0.000003,0.000564,0.000003,20.354324,0.667670,0.220908,0.000002,-3.708254e-09,0.000002,0.0,1630.0,209.0,0.562039,0.000127,1640.0,0.695498,1.708365,9999.000000,-3.306952e-10,0.018259,8.325051e-08,447.0,0
506633,0.000003,0.000561,0.000003,19.680088,0.665403,0.256714,0.000002,3.998199e-09,0.000002,0.0,1671.0,225.0,0.561243,0.000120,1676.0,0.717898,1.748904,9999.000000,5.074985e-10,0.019666,7.843683e-08,429.0,0
506634,0.000003,0.000507,0.000003,18.696962,0.655014,0.221881,0.000002,-2.609512e-09,0.000002,0.0,1636.0,230.0,0.562165,0.000128,1622.0,0.725297,1.808618,9999.000000,3.268887e-10,0.019889,6.604886e-08,486.0,0
506635,0.000005,0.000654,0.000005,17.202044,0.680660,0.293953,0.000002,-7.630152e-11,0.000002,0.0,1685.0,234.0,0.561131,0.000174,1673.0,0.730297,1.704340,9999.000000,-3.416951e-10,0.021868,1.204570e-07,418.0,0


In [152]:
print(df_encoded['Tag'].value_counts())

1    419430
0     87207
Name: Tag, dtype: int64


In [75]:
df_encoded['Tag'].unique()

array([1, 0])

In [76]:

x=df_encoded.drop(["Tag"]  ,axis=1)
x.shape

(506637, 22)

In [77]:
y = df_encoded.loc[:,'Tag'].values
y.shape

(506637,)

In [78]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import ClusterCentroids
clusters = RandomOverSampler(sampling_strategy='minority',random_state =1)#RandomOverSampler => pour le oversampling
clusters.fit(x,y)
X_Sampled,Y_Sampled = clusters.fit_resample(x,y)

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [79]:
X_Sampled.shape

(838860, 22)

In [80]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_Sampled)
X_Sampled = scaler.transform(X_Sampled)

In [81]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X_Sampled, Y_Sampled, test_size = 0.25)

In [82]:
y_test.shape

(209715,)

In [83]:
x_train = np.reshape(x_train, (x_train.shape[0],1,X_Sampled.shape[1]))
x_test = np.reshape(x_test, (x_test.shape[0],1,X_Sampled.shape[1]))

In [84]:
import tensorflow as tf
from tensorflow.keras import Sequential

from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
tf.keras.backend.clear_session()

model = Sequential()
model.add(LSTM(100, input_shape=(1,22),activation="relu",return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(50,activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
from keras.optimizers import SGD
model.compile(loss ='binary_crossentropy', optimizer = "adam", metrics = ['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 1, 100)            49200     
_________________________________________________________________
dropout (Dropout)            (None, 1, 100)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 50)                30200     
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 51        
Total params: 79,451
Trainable params: 79,451
Non-trainable params: 0
_________________________________________________________________


In [85]:
model.fit(x_train, y_train, epochs = 100, validation_data= (x_test, y_test))
score, acc = model.evaluate(x_test, y_test)


Epoch 1/100
19661/19661 [==============================] - 99s 5ms/step - loss: 0.4004 - accuracy: 0.8116 - val_loss: 0.2773 - val_accuracy: 0.8754
Epoch 2/100
19661/19661 [==============================] - 89s 5ms/step - loss: 0.2831 - accuracy: 0.8718 - val_loss: 0.2525 - val_accuracy: 0.8850
Epoch 3/100
19661/19661 [==============================] - 88s 4ms/step - loss: 0.2521 - accuracy: 0.8872 - val_loss: 0.2061 - val_accuracy: 0.9104
Epoch 4/100
19661/19661 [==============================] - 87s 4ms/step - loss: 0.2376 - accuracy: 0.8945 - val_loss: 0.1934 - val_accuracy: 0.9180
Epoch 5/100
19661/19661 [==============================] - 87s 4ms/step - loss: 0.2271 - accuracy: 0.8996 - val_loss: 0.1919 - val_accuracy: 0.9168
Epoch 6/100
19661/19661 [==============================] - 85s 4ms/step - loss: 0.2193 - accuracy: 0.9035 - val_loss: 0.1770 - val_accuracy: 0.9254
Epoch 7/100
19661/19661 [==============================] - 85s 4ms/step - loss: 0.2122 - accuracy: 0.9072 - val_

In [86]:
model.save("/content/drive/MyDrive/Model/LSTM_Model2.h5")